# Ettoday 網路爬蟲實作練習


* 能夠利用 Request + BeatifulSour 撰寫爬蟲，並存放到合適的資料結構


## 作業目標

根據範例 ，完成以下問題：

* ① 取出今天所有的發文
* ② 如果想要依照類別分類，怎麼儲存會比較好？
* ③ 哪一個類別的文章最多




In [1]:
import time
import requests
from pprint import pprint
from datetime import datetime
from bs4 import BeautifulSoup


def get_articles(soup, crawler_date=datetime.now()):
    """取得頁面中今天的文章，並整理成Dict

        Args:
            soup: BeautifulSoup物件
            crawler_date: 爬取日期(預設今日)
        Returns:
            articles: 爬取到的文章
            is_continue: 是否尚未結束
        """
    is_continue = True
    articles = []
    for article_block in soup.select('h3'):
        time_text = article_block.select_one('.date').text
        time_ = datetime.strptime(time_text, '%Y/%m/%d %H:%M')
        category = article_block.select_one('.tag').text
        title = article_block.select_one('a').text
        url_ = 'https://www.ettoday.net' + article_block.select_one('a').get('href')
        if time_.date() == crawler_date.date():
            articles.append(
                {
                    '時間': time_,
                    '類別': category,
                    '標題': title,
                    'URL': url_
                }
            )
        else:
            is_continue = False
            break
    return articles, is_continue

### ① 取出今天所有的發文

In [2]:
url = 'https://www.ettoday.net/news/news-list.htm'
r = requests.get(url)

soup = BeautifulSoup(r.text, 'html.parser')
soup = soup.select_one('.part_list_2')

articles, _ = get_articles(soup)

print(f'共有 {len(articles)} 則新聞')
print('以下顯示最新一則：')
pprint(articles[0])

共有 100 則新聞
以下顯示最新一則：
{'URL': 'https://www.ettoday.net/news/20200614/1737471.htm',
 '時間': datetime.datetime(2020, 6, 14, 13, 49),
 '標題': '中職二軍／相隔612天登板\u3000林國裕先發13球失1分',
 '類別': '體育'}


In [3]:
# 取得真正的"今天所有的發文"
# 發現到 ETtoday 的此頁面只會載入前100則新聞
# 後續採動態載入的方式

url = "https://www.ettoday.net/show_roll.php"

date_text = input('請輸入想爬取的日期(格式"2020/06/12"，預設今日)：').strip()
if date_text:
    try:
        crawler_date = datetime.strptime(date_text, '%Y/%m/%d')
    except ValueError:
        print('輸入日期格式錯誤!')
        crawler_date = datetime.now()
else:
    crawler_date = datetime.now()
print('——————————————————————————————')

date_text = crawler_date.strftime('%Y%m%d')
all_articles = []
is_continue = True
index = 1

# 如果新聞都還是今日的，持續爬取
while is_continue:
    payload = f"offset={index}&tPage=3&tFile={date_text}.xml"
    headers = {'content-type': "application/x-www-form-urlencoded"}
    r = requests.post(url, data=payload, headers=headers)

    soup = BeautifulSoup(r.text, 'html.parser')

    articles, is_continue = get_articles(soup, crawler_date)
    all_articles.extend(articles)
    index += 1
    # 最多爬取50次(500篇新聞)
    # if index > 50:
    #     break
    # 加入延遲，盡量不要造成對方伺服器的負擔
    time.sleep(.5)

print(f'共有 {len(all_articles)} 則新聞')
print('以下顯示最新一則：')
pprint(all_articles[0])

請輸入想爬取的日期(格式"2020/06/12"，預設今日)：2020/06/13
——————————————————————————————
共有 578 則新聞
以下顯示最新一則：
{'URL': 'https://www.ettoday.net/news/20200613/1737320.htm',
 '時間': datetime.datetime(2020, 6, 13, 23, 54),
 '標題': '快訊／台鐵深夜出事！「太魯閣號」彰化平交道碰撞\u3000雙向延誤至少3小時',
 '類別': '社會'}


### ② 如果想要依照類別分類，怎麼儲存會比較好？

In [4]:
all_article_category = {}
for article in all_articles:
    if article['類別'] in all_article_category:
        all_article_category[article['類別']].append(article)
    else:
        all_article_category[article['類別']] = [article]

# 用Dict依照不同分類儲存
# print(all_article_category)
# 因資料量大，此處只顯示'體育'分類
print(all_article_category.get('體育', {}))

[{'時間': datetime.datetime(2020, 6, 13, 22, 24), '類別': '體育', '標題': '企聯女壘／一個月拿7次勝投！\u3000旺獅美洋將助隊衝排名', 'URL': 'https://www.ettoday.net/news/20200613/1737301.htm'}, {'時間': datetime.datetime(2020, 6, 13, 21, 57), '類別': '體育', '標題': '中職／被猿打11安仍奪跨季8連勝\u3000萊福力感謝隊友', 'URL': 'https://www.ettoday.net/news/20200613/1737292.htm'}, {'時間': datetime.datetime(2020, 6, 13, 21, 47), '類別': '體育', '標題': 'WSBL／林文佑本季大躍進\u3000進步獎熱門人選', 'URL': 'https://www.ettoday.net/news/20200613/1737294.htm'}, {'時間': datetime.datetime(2020, 6, 13, 21, 38), '類別': '體育', '標題': '中職／吳俊偉抗壓飆K退場哭了\u3000周思齊鼓勵：投得很棒', 'URL': 'https://www.ettoday.net/news/20200613/1737290.htm'}, {'時間': datetime.datetime(2020, 6, 13, 21, 33), '類別': '體育', '標題': '中職／吳俊偉不穩差點換投\u3000丘總：我們要與壓力抗衡', 'URL': 'https://www.ettoday.net/news/20200613/1737287.htm'}, {'時間': datetime.datetime(2020, 6, 13, 21, 28), '類別': '體育', '標題': '中職／跑出三壘打還達成300二壘打\u3000周思齊：腳程不錯吧', 'URL': 'https://www.ettoday.net/news/20200613/1737273.htm'}, {'時間': datetime.datetime(2020, 6, 13, 21, 2

### ③ 哪一個類別的文章最多

In [5]:
max_category = ''
max_length = 0
for _category, articles in all_article_category.items():
    print(_category, len(articles))
    if len(articles) > max_length:
        max_category = _category
        max_length = len(articles)

print('——————————————————————————————')
print(f'最多文章的類別是：{max_category}')
print(f'共有 {max_length} 篇 / 全部 {len(all_articles)} 篇')

社會 55
影劇 75
財經 8
健康 19
大陸 29
時尚 25
生活 79
體育 83
政治 50
ET車雲 8
地方 33
國際 42
旅遊 28
軍武 3
房產雲 15
保險 4
寵物動物 11
3C家電 2
消費 2
探索 5
論壇 2
——————————————————————————————
最多文章的類別是：體育
共有 83 篇 / 全部 578 篇
